In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

bats_data = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/products/Bats_cmems.zarr", engine="zarr")
bats_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
bats_data.time.attrs["axis"] = "T"
bats_data = bats_data.sel(time=slice(time_start, time_end))
_ = bats_data.load()

cafe_npp = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/products/Bats_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.latitude.attrs = {"units": "degrees_north", "long_name": "latitude", "axis": "Y"}
cafe_npp.longitude.attrs = {"units": "degrees_east", "long_name": "longitude", "axis": "X"}
# cafe_npp = cafe_npp.rename({"lat": "latitude", "lon": "longitude"})
_ = cafe_npp.load()

In [3]:
bats_data

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 8767, depth: 3)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 210kB 20....
    U                    (latitude, longitude, time, depth) float64 210kB 0.0...
    V                    (latitude, longitude, time, depth) float64 210kB -0....
    mnkc_epi             (latitude, longitude, time) float64 70kB 0.7326 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 70kB 0.6836 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 70kB 1.118 ... 1...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 70kB 0.579 ... 1...
    mnkc_umeso           (latitude, longitude, time) float64 70kB 0.3336 ... ...
    npp                  (latitude, longitude, time) float64 70kB 450.0 ... 2...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 210kB 100...
    zeu                  (latitude, longitude, time) float64 70kB 66.78 ... 7...
    zooc                 (latitude, longitude, time) float64 70kB 0.3742 ... ...

In [4]:
cafe_npp

<xarray.Dataset> Size: 140kB
Dimensions:    (time: 8767, latitude: 1, longitude: 1)
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 70kB 1998-01-01 1998-01-02 ... 2022-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 70kB 262.5 257.1 ... 228.4
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [5]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../../seapopym-data/data/zooplankton_product/Bats_zooplankton.nc")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)
observations = [obs_bats]

Create structure for SeapoPym simulation.


In [6]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=bats_data["T"], resolution=1 / 12),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12),
)

2024-11-21 11:24:41,124 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [7]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        tr_max=Parameter("D1N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        tr_max=Parameter("D2N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [8]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [ ]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    ETA=1,
    INDPB=0.1,
    CXPB=0.5,
    MUTPB=0.2,
    NGEN=5,
    POP_SIZE=10,
    cost_function_weight=(-1,),
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [10]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [11]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61381 instead
  warnings.warn(


And watch the magic on the Dask dashboard :


In [12]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:61381/status,
Dashboard: http://127.0.0.1:61381/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61382,Workers: 4
Dashboard: http://127.0.0.1:61381/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:61393,Total threads: 2
Dashboard: http://127.0.0.1:61400/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:61385,


And execute the process.


In [13]:
viewer = genetic_algo.optimize()

2024-11-21 11:24:42,704 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 11:24:42,708 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 11:24:42,712 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 11:24:42,712 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 11:24:42,715 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 11:24:42,715 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 11:24:42,718 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-21 11:24:42,723 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-21 11:24:42,724 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-21 11:24:42,754 :: Seapodym ::  DEBUG ::
|	Direct computation for _average_temperature.

2024-11-21 11:24:42,762 :: Seapod

Finaly here is the result :


In [14]:
viewer.logbook

,gen,nevals,avg,std,min,max,nvalide,ninvalide
0,0,10,8.852038e-06,1.915979e-05,1.388461e-07,5.169416e-05,6,4
1,1,7,3.365392e-07,8.886897e-08,1.388461e-07,4.304619e-07,8,2
2,2,5,2.633433e-07,1.073315e-07,1.388461e-07,4.198998e-07,10,0
3,3,7,9.080509e-07,2.203398e-06,1.388461e-07,7.515611e-06,10,0
4,4,4,1.609957e-07,6.644878e-08,1.388461e-07,3.603420e-07,10,0
5,5,5,1.496308e-07,3.235398e-08,1.388461e-07,2.466927e-07,10,0


In [15]:
viewer.box_plot(5)

In [16]:
viewer.parallel_coordinates()